<a href="https://colab.research.google.com/github/RIW3n/Degrees/blob/main/fraud_detec_zindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import PoissonRegressor
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler,RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,VotingRegressor
from sklearn.metrics import classification_report,mean_squared_log_error
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import VotingClassifier,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE,KMeansSMOTE,SVMSMOTE

In [3]:
ct = pd.read_csv('/content/client_train.csv')
it = pd.read_csv('/content/invoice_train.csv')

In [4]:

ct.head()

,disrict,client_id,client_catg,region,creation_date,target
0,60,train_Client_0,11,101,31/12/1994,0.0
1,69,train_Client_1,11,107,29/05/2002,0.0
2,62,train_Client_10,11,301,13/03/1986,0.0
3,69,train_Client_100,11,105,11/07/1996,0.0
4,62,train_Client_1000,11,303,14/10/2014,0.0


In [5]:
it.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0.0,0.0,14302.0,14384.0,4.0,ELEC
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0.0,0.0,12294.0,13678.0,4.0,ELEC
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,0.0,0.0,14624.0,14747.0,4.0,ELEC
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,0.0,0.0,14747.0,14849.0,4.0,ELEC
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,0.0,0.0,15066.0,15638.0,12.0,ELEC


In [6]:
df = pd.merge(ct,it,on='client_id',how='inner')

In [7]:
le = LabelEncoder()

In [8]:
df['counter_type'] = le.fit_transform(df['counter_type'])

In [9]:
aggregated_data = df.groupby('client_id').agg({'disrict': 'mean', 'client_catg': 'mean', 'region': 'mean', 'tarif_type': 'mean', 'counter_number': 'mean', 'counter_code': 'mean', 'consommation_level_2': 'mean', 'consommation_level_3': 'mean', 'consommation_level_4': 'mean', 'counter_type': pd.Series.mode,'target':pd.Series.mode})

In [10]:
aggregated_data.head(10)

,disrict,client_catg,region,tarif_type,counter_number,counter_code,consommation_level_2,consommation_level_3,consommation_level_4,counter_type,target
client_id,,,,,,,,,,,
train_Client_0,60.0,11.0,101.0,11.000000,1.335667e+06,203.685714,10.571429,0.000000,0.000000,0,0.0
train_Client_1,69.0,11.0,107.0,11.000000,6.789020e+05,203.000000,0.000000,0.000000,0.000000,0,0.0
train_Client_10,62.0,11.0,301.0,11.000000,5.727650e+05,203.222222,37.888889,0.000000,0.000000,0,0.0
train_Client_100,69.0,11.0,105.0,11.000000,2.078000e+03,413.000000,0.000000,0.000000,0.000000,0,0.0
train_Client_1000,62.0,11.0,303.0,11.000000,1.957500e+04,207.000000,104.857143,117.357143,36.714286,0,0.0
train_Client_10000,69.0,11.0,103.0,22.479167,8.149729e+05,124.625000,10.437500,3.770833,0.000000,0,0.0
train_Client_100000,62.0,11.0,309.0,25.500000,4.446350e+04,209.000000,6.550000,0.000000,0.000000,"[0, 1]",0.0
train_Client_100001,60.0,11.0,101.0,25.128205,5.904181e+05,106.538462,0.923077,0.000000,0.000000,0,0.0
train_Client_100002,62.0,11.0,301.0,11.000000,1.441290e+05,413.000000,0.000000,0.000000,0.000000,0,0.0


In [11]:
x = aggregated_data.drop('target',axis=1)

In [12]:
x.head()

,disrict,client_catg,region,tarif_type,counter_number,counter_code,consommation_level_2,consommation_level_3,consommation_level_4,counter_type
client_id,,,,,,,,,,
train_Client_0,60.0,11.0,101.0,11.0,1335667.0,203.685714,10.571429,0.000000,0.000000,0
train_Client_1,69.0,11.0,107.0,11.0,678902.0,203.000000,0.000000,0.000000,0.000000,0
train_Client_10,62.0,11.0,301.0,11.0,572765.0,203.222222,37.888889,0.000000,0.000000,0
train_Client_100,69.0,11.0,105.0,11.0,2078.0,413.000000,0.000000,0.000000,0.000000,0
train_Client_1000,62.0,11.0,303.0,11.0,19575.0,207.000000,104.857143,117.357143,36.714286,0


In [13]:
y = aggregated_data['target']

In [14]:
sc = StandardScaler()

In [15]:
x = x.drop('counter_type',axis= 1)

In [16]:
x_scaled = sc.fit_transform(x)

In [17]:
x_scaled

array([[-1.0406902 , -0.10255245, -0.99868397, ..., -0.2198317 ,
        -0.05878244, -0.13651292],
       [ 1.64728541, -0.10255245, -0.94133273, ..., -0.2445856 ,
        -0.05878244, -0.13651292],
       [-0.44336228, -0.10255245,  0.91302418, ..., -0.15586551,
        -0.05878244, -0.13651292],
       ...,
       [-0.14469833, -0.10255245,  1.01816812, ...,  0.70375669,
        -0.05878244, -0.13651292],
       [-0.14469833, -0.10255245, -0.99868397, ..., -0.21688284,
        -0.05174892, -0.13651292],
       [-0.14469833, -0.10255245,  1.00860958, ..., -0.24349649,
        -0.05878244, -0.13651292]])

In [18]:
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y,random_state=2,test_size=0.3,stratify=y)

In [19]:
y_train.value_counts()

target
0.0    2142
1.0     136
Name: count, dtype: int64

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc,roc_curve

In [37]:
lr = LogisticRegression(random_state=21,class_weight="balanced")

In [38]:
ac = AdaBoostClassifier(n_estimators=21, algorithm="SAMME", random_state=0)

In [39]:
bc = BaggingClassifier(estimator=SVC(),n_estimators=11,random_state=0)

In [40]:
dtc = DecisionTreeClassifier(random_state=1,max_depth=5)
knn = KNeighborsClassifier()

In [41]:
vc = VotingClassifier(estimators=[('logistic',lr),
                                  ('tree',dtc),
                                  ('neigbour',knn),
                                  ('adaboost',ac),
                                  ('bagging',bc)])

In [42]:
sampler = SMOTE(sampling_strategy=0.6, random_state=2)

In [43]:
x_resampled,y_resampled = sampler.fit_resample(x_train,y_train)

In [50]:
y_resampled.value_counts()

target
0.0    2142
1.0    1285
Name: count, dtype: int64

In [44]:
vc.fit(x_resampled,y_resampled)

VotingClassifier(estimators=[('logistic',
                              LogisticRegression(class_weight='balanced',
                                                 random_state=21)),
                             ('tree',
                              DecisionTreeClassifier(max_depth=5,
                                                     random_state=1)),
                             ('neigbour', KNeighborsClassifier()),
                             ('adaboost',
                              AdaBoostClassifier(algorithm='SAMME',
                                                 n_estimators=21,
                                                 random_state=0)),
                             ('bagging',
                              BaggingClassifier(estimator=SVC(),
                                                n_estimators=11,
                                                random_state=0))])

In [45]:
vc.score(x_test,y_test)

0.9027635619242579

In [46]:
y_pred = vc.predict(x_test)

In [47]:
fpr, tpr, thresholds = roc_curve(y_test,y_pred)

In [48]:
auc(fpr,tpr)

0.6171719635285732

In [49]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.95      0.94      0.95       919
         1.0       0.24      0.29      0.26        58

    accuracy                           0.90       977
   macro avg       0.60      0.62      0.61       977
weighted avg       0.91      0.90      0.91       977

